# Construct Datasets for benchmarking tasks

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-10-08  

## Dataset1. Enzyme None-enzyme Dataset
The enzyme dataset is consists of two parts: a training set and a testing set. The training set is from snapshot Feb-2018 and excludes those deleted items in snapshot Jun-2020. The training set is consists of 467,973 records, of which 222,290 are enzymes, and 245,683 are not enzymes. The testing set is from snapshot Jun-2020 and excludes these items that appeared in snapshot Feb-2018. The testing set is consists of 8033 records, of which 3579 are enzymes, and 4454 are not-enzymes. Unlike previous works,  we did not filter any sequences in terms of length and homology to make the data more inclusive. We make a label for each sequence, 1 for enzyme and 0 for not-enzyme. 

## Dataset2. Enzyme Quantity Dataset
The enzyme quantity dataset only contains enzyme data, contain 13,108 records. The function quantity ranges from 2 to 10.

## Dataset 3: EC number Dataset

Similar to the enzyme quantity dataset, the EC number dataset is consists of 225,221 enzyme records, 221,642 are training-set, and the rest 3579 are testing-set, covering 4852 EC numbers. Up to Feb 2020, there still exist 267 EC numbers that the model can not handle in the benchmarking. Thus, we exclude the sequences with these 267 EC numbers in the evaluation process. But, this problem can be resolved in the production scenario because we use the entire data from Swiss-Prot. Now the EC coverage is 5307 and can be automatically extended, for the training is real-time based on the publication of Swiss-Prot every 8 weeks. 

## 1. Import packages

In [15]:
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
import config as cfg
from functools import reduce
import matplotlib.pyplot as plt

sys.path.append("./tools/")
import funclib
import exact_ec_from_uniprot as exactec
import minitools as mtool
import embedding_esm as esmebd
import embedding_unirep as unirep

from pandarallel import pandarallel # 导入pandaralle
pandarallel.initialize() # 初始化该这个b...并行库


%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Define Functions

In [1]:
# install axel for download dataset
def install_axel():
    isExists = !which axel
    if 'axel' in str(isExists[0]):
        return True
    else:
        !sudo apt install axel

# add missing '-' for ec number
def refill_ec(ec):   
    if ec == '-':
        return ec
    levelArray = ec.split('.')
    if  levelArray[3]=='':
        levelArray[3] ='-'
    ec = '.'.join(levelArray)
    return ec

def specific_ecs(ecstr):
    if '-' not in ecstr or len(ecstr)<4:
        return ecstr
    ecs = ecstr.split(',')
    if len(ecs)==1:
        return ecstr
    
    reslist=[]
    
    for ec in ecs:
        recs = ecs.copy()
        recs.remove(ec)
        ecarray = np.array([x.split('.') for x in recs])
        
        if '-' not in ec:
            reslist +=[ec]
            continue
        linearray= ec.split('.')
        if linearray[1] == '-':
            #l1 in l1s and l2 not empty
            if (linearray[0] in  ecarray[:,0]) and (len(set(ecarray[:,0]) - set({'-'}))>0):
                continue
        if linearray[2] == '-':
            # l1, l2 in l1s l2s, l3 not empty
            if (linearray[0] in  ecarray[:,0]) and (linearray[1] in  ecarray[:,1]) and (len(set(ecarray[:,2]) - set({'-'}))>0):
                continue
        if linearray[3] == '-':
            # l1, l2, l3 in l1s l2s l3s, l4 not empty
            if (linearray[0] in  ecarray[:,0]) and (linearray[1] in  ecarray[:,1]) and (linearray[2] in  ecarray[:,2]) and (len(set(ecarray[:,3]) - set({'-'}))>0):
                continue
                
        reslist +=[ec]
    return ','.join(reslist)

#format ec
def format_ec(ecstr):
    ecArray= ecstr.split(',')
    ecArray=[x.strip() for x in ecArray] #strip blank
    ecArray=[refill_ec(x) for x in ecArray] #format ec to full
    ecArray = list(set(ecArray)) # remove duplicates
    
    return ','.join(ecArray)

## 3. Download rawdata from unisprot

In [4]:
# download location ./tmp
install_axel()
!axel -n 10 https://ftp.uniprot.org/pub/databases/uniprot/previous_major_releases/release-2018_02/knowledgebase/uniprot_sprot-only2018_02.tar.gz -o ./tmp/uniprot_sprot-only2018_02.tar.gz -q -c
!axel -n 10 https://ftp.uniprot.org/pub/databases/uniprot/previous_major_releases/release-2020_06/knowledgebase/uniprot_sprot-only2020_06.tar.gz -o ./tmp/uniprot_sprot-only2020_06.tar.gz -q -c

No state file, cannot resume!
No state file, cannot resume!


In [ ]:
!axel -n 10 https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_trembl.dat.gz -o ./tmp/uniprot_trembl_latest.dat.gz -q -c

## 4. Extract records from rawdata

In [3]:
# 2018 data
! tar -zxvf ./tmp/uniprot_sprot-only2018_02.tar.gz -C ./tmp/
! mv ./tmp/uniprot_sprot.dat.gz ./tmp/sprot2018.data.gz
! rm -rf ./tmp/*.fasta.gz ./tmp/*.xml.gz

# 2020 data
! tar -zxvf ./tmp/uniprot_sprot-only2020_06.tar.gz -C ./tmp/
! mv ./tmp/uniprot_sprot.dat.gz ./tmp/sprot2020.data.gz
! rm -rf ./tmp/*.fasta.gz ./tmp/*.xml.gz

uniprot_sprot.dat.gz
uniprot_sprot.fasta.gz
uniprot_sprot_varsplic.fasta.gz
uniprot_sprot.xml.gz
uniprot_sprot.dat.gz
uniprot_sprot.fasta.gz
uniprot_sprot_varsplic.fasta.gz
uniprot_sprot.xml.gz


In [27]:
exactec.run_exact_task(infile=cfg.TEMPDIR+'sprot2018.data.gz', outfile=cfg.DATADIR+'sprot2018.tsv')
exactec.run_exact_task(infile=cfg.TEMPDIR+'sprot2020.data.gz', outfile=cfg.DATADIR+'sprot2020.tsv')

556825it [03:41, 2516.40it/s]

finished use time 221.339 s


## 5. Load records

In [3]:
#加载数据并转换时间格式
sprot2018 = pd.read_csv(cfg.DATADIR+'sprot2018.tsv', sep='\t',header=0) #读入文件
sprot2018 = mtool.convert_DF_dateTime(inputdf = sprot2018)


sprot2020 = pd.read_csv(cfg.DATADIR+'sprot2020.tsv', sep='\t',header=0) #读入文件
sprot2020 = mtool.convert_DF_dateTime(inputdf = sprot2020)


## 6. Preprocessing
### 6.1 Drop Duplicates

In [6]:
sprot2018.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2018.reset_index(drop=True, inplace=True)
sprot2020.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot2020.reset_index(drop=True, inplace=True)

### 6.2 format EC

In [13]:
#sprot2018
sprot2018['ec_number'] = sprot2018.ec_number.parallel_apply(lambda x: format_ec(x))
sprot2018['ec_number'] = sprot2018.ec_number.parallel_apply(lambda x: specific_ecs(x))
sprot2018['functionCounts'] = sprot2018.ec_number.parallel_apply(lambda x: 0 if x=='-'  else len(x.split(',')))

#sprot2020
sprot2020['ec_number'] = sprot2020.ec_number.parallel_apply(lambda x: format_ec(x))
sprot2020['ec_number'] = sprot2020.ec_number.parallel_apply(lambda x: specific_ecs(x))
sprot2020['functionCounts'] = sprot2020.ec_number.parallel_apply(lambda x: 0 if x=='-'  else len(x.split(',')))

### 6.3 Split Tain Test

In [6]:
train = sprot2018.iloc[:,np.r_[0,2:8,10:12]]
test = sprot2020.iloc[:,np.r_[0,2:8,10:12]]
test =test[~test.seq.isin(train.seq)]
test.reset_index(drop=True, inplace=True)

### 6.4 Remove changed seqence in test set

In [7]:
test = test[~test.id.isin(test.merge(train, on='id', how='inner').id.values)]
test.reset_index(drop=True, inplace=True)

### 6.4 Trim string

In [8]:
with pd.option_context('mode.chained_assignment', None):
    train.ec_number = train.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    train.seq = train.seq.parallel_apply(lambda x : str(x).strip()) #seq trim
    
    test.ec_number = test.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    test.seq = test.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

In [16]:
test

,id,isenzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,seq,seqlength
0,P69031,False,False,0,-,0,2005-02-01,MAFLKKSLFLVLFLGLVSLSICEQEKREEENQEEDEENEAASEEKR...,71
1,P69019,False,False,0,-,0,2005-02-01,MAFLKKSLFLVLFLGLVSLSICEKEKRQNEEDEDENEAANHEEGSE...,72
2,Q5RF96,True,False,1,3.4.-.-,2,2005-08-30,MARGGDTGCTGPSETSASGAVAIAFPGLEGPPADAQYQTLALTVPK...,169
3,Q9UM00,False,False,0,-,0,2006-06-27,MPRKRKCDLRAVRVGLLLGGGGVYGSRFRFTFPGCRALSPWRVRVQ...,239
4,P0C250,False,False,0,-,0,2006-11-28,MGKLTILVLVAAVLLSTQVMVQGDGDQPADRNAVRRDDNPGGTRGR...,63
...,...,...,...,...,...,...,...,...,...
7096,A0A061ACU2,False,False,0,-,0,2020-12-02,MTVPPLLKSCVVKLLLPAALLAAAIIRPSFLSIGYVLLALVSAVLP...,2442
7097,Q8GYS8,False,False,0,-,0,2020-12-02,MKIGVVLVLLTVFVVVMSSTSVSAQSDEDECLKETGQMQLNCFPYL...,116
7098,Q6NRV0,True,False,1,2.3.2.27,4,2020-12-02,MPIRAYCTICSDFFDNARDVAAITCGHTFHQECLLQWFHSAPHRTC...,464
7099,C5DLH0,True,True,1,3.6.1.-,3,2020-12-02,MGLSADLFVQKRSAASSLKQPKELGFYSKTQEGQFLVNDSSKLSYY...,383


### 6.5 Save train test

In [14]:
train.to_feather(cfg.DATADIR + 'train.feature')
test.to_feather(cfg.DATADIR + 'test.feature')

### 6.6 ESM embedding 

In [11]:
# !pip install fair-esm
tr_rep0, tr_rep32, tr_rep33 = esmebd.get_rep_multi_sequence(sequences=train, model='esm1b_t33_650M_UR50S',seqthres=1022)
tr_rep0.to_feather(cfg.DATADIR + 'train_rep0.feather')
tr_rep32.to_feather(cfg.DATADIR + 'train_rep32.feather')
tr_rep33.to_feather(cfg.DATADIR + 'train_rep33.feather')

tst_rep0, tst_rep32, tst_rep33 = esmebd.get_rep_multi_sequence(sequences=test, model='esm1b_t33_650M_UR50S',seqthres=1022)
tst_rep0.to_feather(cfg.DATADIR + 'test_rep0.feather')
tst_rep32.to_feather(cfg.DATADIR + 'test_rep32.feather')
tst_rep33.to_feather(cfg.DATADIR + 'test_rep33.feather')


Transferred model to GPU


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 469134/469134 [4:46:05<00:00, 27.33it/s]


### 6.7 Unirep embedding

In [ ]:
tr_unirep = unirep.getunirep(train, 20)
tst_unirep = unirep.getunirep(test, 20)

tr_unirep.to_feather(cfg.DATADIR + 'train_unirep.feather')
tst_unirep.to_feather(cfg.DATADIR + 'test_unirep.feather')


## 7. Build benchmarking datasets
### 7.1 Task 1 isEnzyme

In [15]:
task1_train = train.iloc[:,np.r_[0,7,1]]
task1_test = test.iloc[:,np.r_[0,7,1]]

task1_train.to_feather(cfg.DATADIR + 'task1/train.feather')
task1_test.to_feather(cfg.DATADIR + 'task1/test.feather')

### 7.2 Task2 Function Counts

In [16]:
task2_train = train[train.functionCounts >0]
task2_train.reset_index(drop=True, inplace=True)
task2_train = task2_train.iloc[:,np.r_[0,7,3]]

task2_test = test[test.functionCounts >0]
task2_test.reset_index(drop=True, inplace=True)
task2_test = task2_test.iloc[:,np.r_[0,7,3]]

task2_train.to_feather(cfg.DATADIR + 'task2/train.feather')
task2_test.to_feather(cfg.DATADIR + 'task2/test.feather')


### 7.3 Task3 EC Number

In [13]:
task3_train = train[train.functionCounts >0]
task3_train.reset_index(drop=True, inplace=True)
task3_train = task3_train.iloc[:,np.r_[0,7,4]]

task3_test = test[test.functionCounts >0]
task3_test.reset_index(drop=True, inplace=True)
task3_test = task3_test.iloc[:,np.r_[0,7,4]]

task3_train.to_feather(cfg.DATADIR + 'task3/train.feather')
task3_test.to_feather(cfg.DATADIR + 'task3/test.feather')
